In [666]:
'''
stretch = lambda x,F : np.outer(F,x)
shift = lambda x,R : x - R[:,np.newaxis]
wav = lambda x : np.piecewise(x,[x%1 < 0.5], [-1,1])
env = lambda x,L : np.piecewise(x, [x < 0, x > L[:,np.newaxis]], [0,0,1])
merge = lambda x,y : np.sum(x*y,axis=0)
daw_daw = lambda x,F,R,L : merge(env(shift(x,R),L),wav(stretch(x,F)))
'''

'\nstretch = lambda x,F : np.outer(F,x)\nshift = lambda x,R : x - R[:,np.newaxis]\nwav = lambda x : np.piecewise(x,[x%1 < 0.5], [-1,1])\nenv = lambda x,L : np.piecewise(x, [x < 0, x > L[:,np.newaxis]], [0,0,1])\nmerge = lambda x,y : np.sum(x*y,axis=0)\ndaw_daw = lambda x,F,R,L : merge(env(shift(x,R),L),wav(stretch(x,F)))\n'

In [802]:
import numpy as np
import IPython.display as ipd   # using ipd.Audio and ipd.Display to play sounds
from fractions import Fraction

fs = 48000
pi = np.pi

wav = lambda x : np.sin(2*pi*x)
env = lambda x : np.exp(-0.5*x)

In [787]:
def bank(f,long,wav_block,prints = False, graph = False):
    
    f_frac = Fraction(f).limit_denominator(20)
    cycle_frac = Fraction(fs,f_frac)
    cycle_samps = cycle_frac.numerator
    cycle_periods = cycle_frac.denominator
    
    max_samps = round(fs*long)
    unique_samps = min(max_samps,cycle_samps)
    
    samp_domain = np.arange(unique_samps)*float(f_frac)%fs
    
    ref_low = np.floor(samp_domain).astype(int)
    ref_high = (ref_low + 1)%fs
    decimal = samp_domain%1.0
    
    low_samps = np.take(wav_block,ref_low)
    high_samps = np.take(wav_block,ref_high)
    
    wav_chunk = low_samps*(1-decimal) + high_samps*decimal  
    
    if prints:
        print('\n f: %s \n' % f)
        print('f_frac: %s' % f_frac)
        print('samples per cycle: %s' % cycle_samps)
        print('periods per cycle: %s' % cycle_periods)
        print('max samps: %s' % max_samps)  
    
    if cycle_samps < max_samps:
        tiles = max_samps // cycle_samps + 1
        if prints:
            print('tiles: %s' % tiles)
        wav_chunk = np.tile(wav_chunk,tiles)[:max_samps]
           
    return wav_chunk

def initialize_wav_bank(Fs,longs,prints = False):
    
    wav_block = wav(np.arange(0,1,1/fs))
    wav_bank = [bank(f,l,wav_block,prints) for f,l in zip(Fs,longs)]
    
    return wav_bank

In [ ]:
def initialize_env_block(FRL):
    
    max_L = max(FRL[:,2])
    env_domain = np.arange(0,max_L,1/fs)
    env_block = env(env_domain)
    
    return env_block

In [ ]:
def process_ins(kwargs):
    
    F = kwargs['F']  
    keys = ['F','R','L','tempo','prints']
    values = [F,list(range(len(F))),[1]*len(F),120,False]    
    default_dict = dict(zip(keys, values)) 
    
    for key in default_dict:
        if key in kwargs:
            default_dict[key] = kwargs[key]
    
    return default_dict
    

def create_out(FRL,wav_env_bank,F_key):

    starts = np.round(fs*FRL[:,1]).astype(int)
    durs = np.round(fs*FRL[:,2]).astype(int)
    ends = starts + durs
    
    total_duration = max(ends)
    out = np.zeros(total_duration)
    
    num_notes = FRL.shape[0]
    
    for k in range(num_notes):
        out[ starts[k] : ends[k] ] += wav_env_bank[F_key[k]][:durs[k]]
        
    return out

In [803]:
def main(**kwargs):
            
    inputs = process_ins(kwargs)
    
    F = inputs['F']
    tempo = inputs['tempo']
    R = np.array(inputs['R'])/tempo*60
    L = np.array(inputs['L'])/tempo*60
    prints = inputs['prints']
    
    FRL = np.array(list(zip(F,R,L)))
    
    F_unique = np.sort(np.array(list(set(F))))
    F_key = [np.where(F_unique == f)[0][0] for f in F]
    
    long_note = np.array([max(FRL[FRL[:, 0] == f, 2]) for f in F_unique])
    
    wav_bank = initialize_wav_bank(F_unique,long_note,prints)
    env_block = initialize_env_block(FRL)
    
    long_durs = np.round(long_note*fs).astype(int)
    wav_env_bank = [env_block[:s]*w[:s] for w,s in zip(wav_bank,long_durs)]
    
    out = create_out(FRL,wav_env_bank,F_key)
    
    return out

In [823]:
F1 = [800,700,600,500,400,300,200,100]*2
R1 = [0,0.1,0.2,0.3,0,0.1,0.3,0.4] + [1,1.1,1.2,1.3,1,1.1,1.3,1.4]

F2 = list(np.array(F1)*1.75)
R2 = list(np.array(R1)+2)

F3 = F1 + F2
R3 = R1 + R2

F4 = F3[::-1]
R4 = list(np.array(R3) + 4)

R5 = np.array([0]*5 + [0.4] + [0.1] + [0.35]) + 8
R6 = R5[::-1] + 1

R7 = list(R5) + list(R6)
R8 = R7 + list(np.array(R7) + 2)
R9 = R8 + list(np.array(R8) + 4)

R = (R3 + R4 + R9)[::-1]
F = (F3 + F4)*2


L = [1]*len(F)


In [835]:
Fa = 10*np.array([30,36,45,32,40,48,27,32,40,30,36,45,30,30,30,36,36,36,45,45,45,32,32,32,40,40,40,48,48,48])
Ra = [0,0,0,1,1,1,2,2,2,3,3,3,4,4+(1/3),4+(2/3),4,4+(1/3),4+(2/3),4,4+(1/3),4+(2/3),5,5+(1/3),5+(2/3),5,5+(1/3),5+(2/3),5,5+(1/3),5+(2/3)]

In [836]:
out = main(F=Fa,R=Ra,tempo=60)
ipd.display(ipd.Audio(out, rate=fs))  